In [1]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari

In [2]:
os.chdir('/home/ali/packages/s2p-lbm/')

from suite3d.job import Job
from suite3d import lbmio, utils, ui
from suite3d import tiff_utils as tfu
from suite3d import file_utils as flu

In [3]:
subjects_dir =   '/mnt/zortex-subjects/'
expt_info = { 
    'subject':         'TC022',
    'date' :          '2024-08-09',
    'expnum' :         [2,3,4,5],}

tifs, si_params, exp_str = flu.find_exp(subjects_dir, **expt_info, verbose=False)

job_params = {    
    # number of channels recorded in the tiff file, typically 30
    'n_ch_tif' : 26,
    # number of planes in the deeper cavity, typically 15
    'cavity_size' : 13,
    # convert from the Scanimage channel ordering to deep-to-shallow ordering
    # in our case, ScanImage channel numbers are in temporal order. 
    'planes' : n.array([ 0,  2,  4,  6,  8, 10, 12, 14,
                        16, 18, 20, 22, 24, 1,  3, 5,  7,9,11, 13, 15,17,19,21]),
    # number of files to use for the initial pass
    'n_init_files' :   4,
    
    # number of pixels to fuse between the ROI strips
    # the auto-detection doesn't always work well, recommend 
    # manually tuning it to reduce the stitching artifacts
    'fuse_shift_override' : 6,
    
    # will try to automatically estimate crosstalk using 
    # the shallowest crosstalk_n_planes planes. if you want to override,
    # set override_crosstalk = float between 0 and 1
    'subtract_crosstalk' : True,
    
    # volume rate in acquisition
    'fs' : flu.get_si_params(tifs[0])['vol_rate'],
    
    # 3D GPU registration - fast! 
    # for now, 3D + GPU are well-tested, other options might have minor bugs
    # if you run into them, let us know! we'll fix them
    '3d_reg' : False,
    'use_GPU_registration' : True,
}

In [4]:

job_params['fs'] = si_params['vol_rate']

savedir = os.path.join('/mnt/md0/runs', expt_info['subject'], expt_info['date'])
os.makedirs(savedir, exist_ok=True)

In [5]:
# Create the job
job = Job('/mnt/md0/runs',exp_str, tifs = tifs,
          params=job_params, create=False, overwrite=True, verbosity = 3)

   Found and loaded params from /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/params.npy


In [6]:
# optional parameters for initialization
# load 1 file to initialize, and select 200 frames randomly to do initalization on
job.params['n_init_files'] = 4
job.params['init_n_frames'] = None
job.params['init_file_sample_method'] = 'even'
job.params['voxel_size_um'] = (20, 2.5, 2.5)

In [7]:
# %%time
# summary = job.run_init_pass()

In [8]:
# summary = job.load_summary()

In [9]:
# split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [10]:
# %%time
# job.register_gpu_3d()

In [11]:
# mov_full = job.get_registered_movie('registered_fused_data', 'fused')
# im_full = mov_full[:,:200].mean(axis=1).compute()

In [12]:
# job.params['cell_filt_type'] = 'gaussian'
# job.params['npil_filt_xy_um'] = 80
# job.params['sdnorm_exp'] = 0.8
# params_to_sweep = {
#     'cell_filt_xy_um' : (5,7, 10),
#     'npil_filt_xy_um' : (40, 80),
#     'sdnorm_exp' : (0.8,),
# }
# job.sweep_corrmap(params_to_sweep, iter_limit=2)

In [13]:
# %%time
# job.params['voxel_size_um'] = (20, 2.5, 2.5)
# job.params['npil_filt_xy_um'] = 40
# job.params['cell_filt_xy_um'] = 7
# job.params['sdnorm_exp'] = 0.80
# corr_map = job.calculate_corr_map()

In [14]:
res = job.load_corr_map_results()
vmap = res['vmap']

vmap[0] -= (vmap[0].mean() - vmap[1].mean())
vmap[-1] -= (vmap[-1].mean() - vmap[-2].mean())

In [21]:
job.params['peak_thresh'] = 0.39
job.params['detection_timebin'] = 1 
job.params['percentile'] = 95.0

job.params['extend_thresh'] = 0.1
job.params['activity_thresh'] = 7.0
job.params['ext_use_ratio'] = False
job.params['max_iter'] = 100

params_to_sweep = {
    'percentile' : (95.0,),
    'activity_thresh' : (7.0,),
    'extend_thresh' : (0.1, 0.9),
    'ext_use_ratio' : (True, False)
}

sweep = job.sweep_segmentation(params_to_sweep, all_combinations=True, ts = (0,4000), 
                       patches_to_segment = (20,), vmap=vmap)

   Setting up sweep
      Found dir /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/sweeps/seg
      Updating self.dirs tag sweeps-seg
   Total of 4 combinations
      Created directory for comb_00000 with params comb00000-params-percentile_95.000-activity_thresh_7.000-extend_thresh_0.100-ext_use_ratio_1.000
      Found dir /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/sweeps/seg/comb_00000
      Updating self.dirs tag sweeps-seg-comb_00000
      Created directory for comb_00001 with params comb00001-params-percentile_95.000-activity_thresh_7.000-extend_thresh_0.100-ext_use_ratio_0.000
      Found dir /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/sweeps/seg/comb_00001
      Updating self.dirs tag sweeps-seg-comb_00001
      Created directory for comb_00002 with params comb00002-params-percentile_95.000-activity_thresh_7.000-extend_thresh_0.900-ext_use_ratio_1.000
      Found dir /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/sweeps/seg/comb_00002
      Updating self.dirs tag sweeps-seg-comb_00002
   

In [22]:
from suite3d import default_params

In [26]:
job.params['peak_thresh'] = 0.39
job.params['detection_timebin'] = 1 
job.params['percentile'] = 95.0

job.params['extend_thresh'] = 0.1
job.params['activity_thresh'] = 7.0
job.params['ext_use_ratio'] = False
job.params['max_iter'] = 5000

In [27]:
job.segment_rois(vmap=vmap)

      Found dir /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/segmentation
      Updating self.dirs tag segmentation
   Saved a copy of params at /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/segmentation
   Updated main params file
      Found dir /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois
      Updating self.dirs tag rois
   Saving results to /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/segmentation and /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois 
dict_keys(['max_img', 'mean_img', 'vmap', 'all_params'])
   Detecting from patch 1 / 88
      Found dir /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/segmentation/patch-0000
         Loading 6.68 GB movie to memory, shape: (5187, 24, 120, 120) 
         Loaded
         Loading movie patch to shared memory
         Loaded
      Starting extraction with peak_thresh: 0.390 and Th2: 7.000
      Iter 0000: peak is too small  - ending extraction
   Found 0 cells in 0 iterations
   Saving cells to /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/segmentat

'/mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois'

In [28]:
job.compute_npil_masks(stats_dir = job.dirs['rois'])


'/mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois'

In [29]:

traces = job.extract_and_deconvolve(stats_dir=job.dirs['rois'])

   Updated main params file
   Movie shape: (24, 5187, 1047, 751)
20960
   Extracting 20960 valid cells, and saving cell flags to /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois/iscell_extracted.npy
   Extracting activity
Will extract in 11 batches of 500
Saving intermediate results to /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois
Extracting batch 0000 of 0011
Batch size: 35 GB
Extracting batch 0001 of 0011
Batch size: 35 GB
Extracting batch 0002 of 0011
Batch size: 35 GB
Extracting batch 0003 of 0011
Batch size: 35 GB
Extracting batch 0004 of 0011
Batch size: 35 GB
Extracting batch 0005 of 0011
Batch size: 35 GB
Extracting batch 0006 of 0011
Batch size: 35 GB
Extracting batch 0007 of 0011
Batch size: 35 GB
Extracting batch 0008 of 0011
Batch size: 35 GB
Extracting batch 0009 of 0011
Batch size: 35 GB
Extracting batch 0010 of 0011
Batch size: 13 GB
   Deconvolving
   Saving to /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois


In [30]:
job.export_results('/mnt/zeytin-f1/s3d-results/',result_dir_name='rois')

   Created dir /mnt/zeytin-f1/s3d-results/s3d-results-TC022_2024-08-09_2-3-4-5 to export results
      Loading from /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois/stats_small.npy
      Loading from /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois/info.npy
      Loading from /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois/F.npy
      Loading from /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois/spks.npy
      Loading from /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois/Fneu.npy
      Loading from /mnt/md0/runs/s3d-TC022_2024-08-09_2-3-4-5/rois/iscell.npy
   /mnt/zortex-subjects/TC022/2024-08-09/2/2024-08-09_2_TC022_2P_00001_00001.tif is 100 frames and 3971882554 bytes
   /mnt/zortex-subjects/TC022/2024-08-09/2/2024-08-09_2_TC022_2P_00001_00028.tif is 48 frames and 1906524090 bytes
   /mnt/zortex-subjects/TC022/2024-08-09/3/2024-08-09_3_TC022_2P_00001_00004.tif is 32 frames and 1271029178 bytes
   /mnt/zortex-subjects/TC022/2024-08-09/4/2024-08-09_4_TC022_2P_00001_00001.tif is 46 frames a